In [3]:
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box
import random
import matplotlib.pyplot as plt


In [4]:
import time

def change_variable_every_second(x):
    variable_to_change = 0

    while True:
        print("Current value:", variable_to_change)
        variable_to_change += x
        time.sleep(1)


In [15]:
class DamGateControlEnv(Env):
    def __init__(self, num_levels=16):
        self.num_levels = num_levels
        self.total_inflow = random.uniform(-1000000,1000000)
        self.observation_space = Box(low = np.array([0.0]) , high = np.array([445.0]))
        self.action_space = Discrete(num_levels)
        self.state = 438.0 + random.uniform(-1, 1)

    def step(self, action):
        # Update the gate positions based on the action
        if self.state <= 438 and self.state <=438.65:
            reward = 1
        else:
            reward = -1

        self.state += self.total_inflow/
        self.state +=  action
        self.state = np.clip(self.state, 0, self.num_levels - 1)

        # Check if the episode is done (you may need to customize this based on your problem)
        if self.state >438.65:
            done = True
        else:
            done = False

        self.state += random.randint(-1, 1)

        info = {}

        return self.state, reward, done, info

    def render():
        pass

    def reset(self):
        self.state = 438.0 + random(-1, 1, 0.01)
        return self.state



# Q-learning algorithm

In [16]:
def q_learning(env, num_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.1):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_history = []
    q_value_history = []  # New list to store Q-values for a specific state-action pair

    for episode in range(num_episodes):
        state = env.reset()
        state = int(state.item()) if hasattr(state, 'item') else int(state)
        done = False
        total_reward = 0

        while not done:
            if np.random.rand() < epsilon:
                action = env.action_space.sample()  # Exploration
            else:
                action = np.argmax(q_table[state])  # Exploitation

            next_state, reward, done, _ = env.step(action)
            next_state = int(next_state.item()) if hasattr(next_state, 'item') else int(next_state)

            # Q-value update
            q_table[state, action] += alpha * (reward + gamma * np.max(q_table[next_state]) - q_table[state, action])

            state = next_state
            total_reward += reward

        rewards_history.append(total_reward)

    # Plot the rewards and Q-values
    plt.figure(figsize=(12, 6))

    # Plot rewards
    plt.subplot(1, 2, 1)
    plt.plot(rewards_history)
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Training Progress')

    # Plot Q-values for the specified state-action pair
    plt.subplot(1, 2, 2)
    plt.plot(q_value_history)
    plt.xlabel('Episode')
    plt.ylabel('Q-value')
    plt.title(f'Q-values for State-Action Pair {plot_state}')

    plt.tight_layout()
    plt.show()

    return q_table, rewards_history

In [17]:
# Function to visualize Q-values for a specific state-action pair
def visualize_q_values(q_table, state_action_pair):
    plt.plot(q_table[state_action_pair[0], state_action_pair[1]])
    plt.xlabel('Episode')
    plt.ylabel('Q-value')
    plt.title(f'Q-values for State-Action Pair {state_action_pair}')
    plt.show()

# Training the agent

In [18]:
env = DamGateControlEnv()

TypeError: Random.random() takes no arguments (3 given)

In [10]:
trained_q_table, rewards_history = q_learning(env)

NameError: name 'env' is not defined

In [ ]:
visualize_q_values(trained_q_table, (0, 0))

# Testing the trained agent

In [ ]:
state = env.reset()
state = int(state.item()) if hasattr(state, 'item') else int(state)
done = False

In [ ]:
while not done:
    action = np.argmax(trained_q_table[state])
    next_state, reward, done, _ = env.step(action)
    next_state = int(next_state.item()) if hasattr(next_state, 'item') else int(next_state)

    print(f"Current State: {state}, Action: {action}, Next State: {next_state}, Reward: {reward}")

    state = next_state